https://chatgpt.com/share/673b8458-f7e4-800a-8a78-6a36162618e6

In [1]:
from sedona.spark import *
import rasterio
import numpy as np
from shapely.geometry import Point, Polygon

In [ ]:
# spark_master_url = "spark://spark.tithia.scio.services:7077"
spark_master_url = "local[*]"

config = (
    SedonaContext.builder()
    .master(spark_master_url) \
    .appName("RasterProcessingWithSedona") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.memory", "8g") \
    .config("spark.hadoop.fs.s3a.endpoint", "x") \
    .config("spark.hadoop.fs.s3a.access.key", "dsQ") \
    .config("spark.hadoop.fs.s3a.secret.key", "cvxvi") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config(
        'spark.jars.packages',
        'org.apache.sedona:sedona-spark-shaded-3.5_2.12:1.6.1,'
        'org.datasyslab:geotools-wrapper:1.6.1-28.2'
    )
    .getOrCreate()
)

sedona = SedonaContext.create(config)

sc = sedona.sparkContext

# Check if the application is connected to the Spark master
def check_spark_connection():
    try:
        # Access the master URL and application ID
        master_url = sc.master
        app_id = sc.applicationId
        
        print(f"Connected to Spark Master at: {master_url}")
        print(f"Application ID: {app_id}")
        return True
    except Exception as e:
        print("Failed to connect to the Spark cluster.")
        print(f"Error: {e}")
        return False
    
# Run the connection check
if check_spark_connection():
    print("Spark connection successful.")
else:
    print("Failed to connect to Spark. Exiting...")
    spark.stop()  # Stop the Spark session if connection fails
    exit(1)

Connected to Spark Master at: local[*]
Application ID: local-1733609654532
Spark connection successful.


In [3]:
geotiff_df = sedona.read.format("binaryFile").load("s3a://data/BFA_hist_land_cover_subindicator_2001_2010.tif")
geotiff_df.show(2)
geotiff_df.createOrReplaceTempView("binary_raster")

+--------------------+-------------------+-------+--------------------+
|                path|   modificationTime| length|             content|
+--------------------+-------------------+-------+--------------------+
|s3a://data/BFA_hi...|2024-12-07 08:35:02|3258373|[49 49 2A 00 08 0...|
+--------------------+-------------------+-------+--------------------+

